In [1]:
import pandas as pd 
import numpy as np 

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import pandas as pd 
import numpy as np 

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest
from scipy.stats import levene, bartlett, shapiro

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Medidas de correlacion
# -----------------------------------------------------------------------
from itertools import combinations

# Librerías necesarias para la conexión a la base de datos
from sqlalchemy import create_engine  # sqlalchemy es una librería que permite conectarse y trabajar con bases de datos de manera más abstracta y flexible. 'create_engine' permite crear una conexión a bases de datos SQL de diferentes tipos, como MySQL, PostgreSQL, SQLite, etc.
import pymysql

# ver todas las columnas
pd.set_option('display.max_columns', None)
# ver todas las filas
pd.set_option('display.max_rows', None)

In [2]:
def abrir (nombre_archivo):
    '''
    Abre el archivo csv
    '''
    df = pd.read_csv(nombre_archivo, on_bad_lines='skip', index_col=0)
    return df

In [3]:
def eda_basico(df):
    
    '''
    Analiza info del df
    '''

    print('🌷Ejemplo de datos del DF:')
    display(df.head(3))
    display(df.tail(3))
    display(df.sample(3))
    print('________________________________________________________________________________________________________')

    print('🌻Número de Filas:')
    display(df.shape[0])
    print('________________________________________________________________________________________________________')

    print('🌱Número de Columnas:')
    display(df.shape[1])
    print('________________________________________________________________________________________________________')

    print('🌼Información de la tabla:')
    display(df.info())
    print('________________________________________________________________________________________________________')

    print('🌑Nombre de las columnas:')
    display(df.columns)
    print('________________________________________________________________________________________________________')

    print('🍄Descripción de los datos numéricos:')
    display(df.describe().T)
    print('________________________________________________________________________________________________________')

    print('🌋Descripción de los datos no-numéricos:')
    display(df.describe(include='object').T)
    print('________________________________________________________________________________________________________')

    print('🍂Saber si hay datos únicos:')
    display(df.nunique())
    print('________________________________________________________________________________________________________')

    print('🐖Que datos son nulos por columnas:')
    display(df.isnull().sum())
    print('________________________________________________________________________________________________________')

    print('🐲Filas duplicadas:')
    total_duplicados = df.duplicated().sum()
    if total_duplicados > 0:
        print(f'cantidad de duplicados: {total_duplicados}')
        print('Primeros duplicados')
        display(df[df.duplicated()].head(3))
    else:
        print('No hay duplicados')
    print('________________________________________________________________________________________________________')

    print('🪹 Columnas constantes (solo 1 valor único):')
    constantes = df.columns[df.nunique() <= 1]
    if len(constantes) > 0:
        print(f'{len(constantes)} columnas con 1 valor único:')
        display(constantes)
    else:
        print('No hay columnas constantes')
    print('________________________________________________________________________________________________________')
    
    print('🚀 Valores únicos en columnas categóricas:')
    for col in df.select_dtypes(include='object'):
        print(f'🔸 {col}')
        print('-----------------------------')
        print(df[col].unique())
        print('________________________________________________________________________________________________________')

    print('🧬 Tipos de datos por columna:')
    display(df.dtypes.value_counts())
    print('________________________________________________________________________________________________________')

    return df

In [4]:
def guardar (df, nombre_archivo): 
    df.to_csv(nombre_archivo)

In [5]:
def transform_data(df):
    print("Transformando datos...")
    df = df.dropna()  # Eliminar valores nulos
    df.columns = df.columns.str.lower()  # Renombrar las columnas con minúsculas
    print(df.head())
    return df

In [6]:
# Configuración de la base de datos
host = '127.0.0.1'
user = 'root'
password = 'AlumnaAdalab'
database = 'bd_etl'

In [7]:
def create_db():
    # Conectar a MySQL usando pymysql
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password
    )

    # Crear un cursor
    cursor = connection.cursor()

    # Crear una base de datos si no existe
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
    print("Base de Datos creada exitosamente.")

    # Cerrar la conexión
    connection.close()


In [8]:
def load_data(table_name, df):
    print(f"Cargando datos en la tabla {table_name}...")

    # Crear conexión a MySQL usando SQLAlchemy
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

    # Insertar datos desde el DataFrame en MySQL
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f"Datos insertados en la tabla {table_name} exitosamente.")

In [9]:
def histogramas_variables_num(df, bins=30):
    """
    Genera histogramas para todas las variables numéricas en el DataFrame.

    Parámetros:
    - df: pandas.DataFrame con datos procesados.
    - bins: número de barras en cada histograma (por defecto 30).
    """
    # Seleccionar columnas numéricas
    columnas_num = df.select_dtypes(include=['number']).columns

    for columna in columnas_num:
        plt.figure(figsize=(8, 5))
        plt.hist(df[columna].dropna(), bins=bins, edgecolor='black')
        plt.title(f'Histograma de {columna}')
        plt.xlabel(columna)
        plt.ylabel('Frecuencia')
        plt.grid(True)
        plt.tight_layout()
        plt.show()